In [20]:
import numpy as np
import pandas as pd

color_est = pd.read_csv("../data/color2emotion_rescaling.csv",index_col = False)
emotion_est = pd.read_csv("../result/emotion_estimation_hand_craft.csv")
action_est = pd.read_csv("../result/action_estimation.csv")

In [21]:
#color_est

In [22]:
emotion_est


,감정,에너지,회복,순환,정화
0,행복,0.600757,0.938826,0.574773,0.565452
1,열정,0.927580,1.197198,0.791780,0.801512
2,두려움,0.281754,0.955181,0.115246,0.601674
3,분노,0.728220,0.811142,0.646665,0.734387
4,우울,0.682752,0.451092,0.501903,0.325328
5,슬픔,0.382926,0.922925,0.370103,0.640172


In [23]:
action_est

,감정,에너지,회복,순환,정화
0,명상,0.801500,0.911456,1.178087,0.650311
1,운동,0.729746,0.966807,0.767528,0.648162
2,산책,0.640690,0.922429,0.543703,0.657409
3,수면,0.563481,0.814027,0.476478,0.718575
4,독서,0.652691,0.970863,0.521449,0.510687


In [24]:
import numpy as np
from numpy import dot
from numpy.linalg import norm
from scipy.stats import pearsonr


def pearson_similarity(a, b):
  return np.dot((a - np.mean(a)), (b - np.mean(b))) / ((np.linalg.norm(a - np.mean(a))) * (np.linalg.norm(b - np.mean(b))))

In [25]:
# 감정 -> 효능 <- 해동 시나리오
class emotion2item():
    def __init__(self ,action_df, emotion_df):
        self.action_df = action_df
        self.emothon_df = emotion_df
             
        
    def sim_item_MAE(self, em_idx):
        em_name , * em_est = self.emothon_df.iloc[em_idx].tolist()
        em_est = np.array(em_est)
        sim_dict = {}
        for idx in self.action_df.index:
            act_name , *act_est = self.action_df.iloc[idx]
            act_est = np.array(act_est)
            sim = np.mean(np.abs(em_est - act_est))
            
            sim_dict[act_name] = sim
            
            print(f'{em_name}과 {act_name}의 MAE : {sim}')
        
        sorted_dict = sorted(sim_dict.items(), key= lambda x:x[1])
        print("-"*20)
        print('{}에서 가장 유사한 item은 {}'.format(em_name,sorted_dict[0][0]))    

    def sim_item_Cosine(self, em_idx):
        em_name , * em_est = self.emothon_df.iloc[em_idx].tolist()
        em_est = np.array(em_est)
        sim_dict = {}
        for idx in self.action_df.index:
            act_name , *act_est = self.action_df.iloc[idx]
            act_est = np.array(act_est)
            sim = dot(em_est, act_est)/(norm(em_est)*norm(act_est))
            
            sim_dict[act_name] = sim
            print(f'{em_name}과 {act_name}의 Cosine 유사도 : {sim}')
        
        sorted_dict = sorted(sim_dict.items(),reverse=True, key= lambda x:x[1])
        print("-"*20)
        print('{}에서 가장 유사한 item은 {}'.format(em_name,sorted_dict[0][0]))    

    def sim_item_Pearson(self, em_idx):
        em_name , * em_est = self.emothon_df.iloc[em_idx].tolist()
        em_est = np.array(em_est)
        sim_dict = {}
        for idx in self.action_df.index:
            act_name , *act_est = self.action_df.iloc[idx]
            act_est = np.array(act_est)
            sim = pearson_similarity(em_est, act_est)
            
            sim_dict[act_name] = sim
            print(f'{em_name}과 {act_name}의 피어슨 유사도 : {sim}')
        
        sorted_dict = sorted(sim_dict.items(), reverse=True ,key= lambda x:x[1])
        print("-"*20)
        print('{}에서 가장 유사한 item은 {}'.format(em_name,sorted_dict[0][0]))    

    def sim_item_callback(self, em_idx,callback,reverse):
        em_name , * em_est = self.emothon_df.iloc[em_idx].tolist()
        em_est = np.array(em_est)
        sim_dict = {}
        for idx in self.action_df.index:
            act_name , *act_est = self.action_df.iloc[idx]
            act_est = np.array(act_est)
            sim = callback(em_est, act_est)
            
            sim_dict[act_name] = sim
            print(f'{em_name}과 {act_name}의 callback 유사도 : {sim}')
        
        sorted_dict = sorted(sim_dict.items(), reverse=reverse ,key= lambda x:x[1])
        print("-"*20)
        print('{}에서 가장 유사한 item은 {}'.format(em_name,sorted_dict[0][0]))    


    def max_item_MAE(self, em_idx, n):
        em_name , * em_est = self.emothon_df.iloc[em_idx].tolist()
        em_est = np.array(em_est)
        effs = ['에너지','회복','순환','정화']
        em_dict = {}
        i = 0
        for eff in effs:
            em_dict[eff] = em_est[i]
            i = i + 1
        max_dict = sorted(em_dict.items(), reverse=True,key= lambda x:x[1])[:n]

        sort_em_list = []
        sort_em_est = []
        for x, y in max_dict:
            sort_em_list.append(x)
            sort_em_est.append(y)

        sort_em_df = self.emothon_df[sort_em_list] 
        em_est = np.array(sort_em_df.iloc[em_idx])
        sort_act_df = self.action_df[sort_em_list]
        print(f'{em_name}의 MAX 차원 {sort_em_list}')


        sim_dict = {}
        for idx in sort_act_df.index:
            act_name , *_ = self.action_df.iloc[idx]
            act_est = np.array(sort_act_df.iloc[idx])
            sim = np.mean(np.abs(em_est - act_est))
            sim_dict[act_name] = sim
            print(f'{em_name}과 {act_name}의 MAE : {sim}')
        sorted_dict = sorted(sim_dict.items(), key= lambda x:x[1])
        print("-"*20)
        print('{}에서 가장 유사한 item은 {}'.format(em_name,sorted_dict[0][0]))    

            

    def complete_item(self, em_idx, n):
        pass

In [26]:
# 감정과 가장 유사한 item을 추천 using MAE
XYZ = emotion2item(action_est, emotion_est)
for i in range(0,5):
    XYZ.sim_item_MAE(i)
    print('\n')


행복과 명상의 MAE : 0.22907152275244402
행복과 운동의 MAE : 0.10810887813568115
행복과 산책의 MAE : 0.04483897735675174
행복과 수면의 MAE : 0.10337312519550322
행복과 독서의 MAE : 0.048014911512533814
--------------------
행복에서 가장 유사한 item은 산책


열정과 명상의 MAE : 0.2373326321442923
열정과 운동의 MAE : 0.1514565795660019
열정과 산책의 MAE : 0.23845964421828586
열정과 수면의 MAE : 0.286376953125
열정과 독서의 MAE : 0.26559477796157205
--------------------
열정에서 가장 유사한 item은 운동


두려움과 명상의 MAE : 0.41873736741642165
두려움과 운동의 MAE : 0.28959708474576473
두려움과 산책의 MAE : 0.21896993927657607
두려움과 수면의 MAE : 0.22525384587546193
두려움과 독서의 MAE : 0.2209525586416324
--------------------
두려움에서 가장 유사한 item은 산책


분노과 명상의 MAE : 0.19727316002051043
분노과 운동의 MAE : 0.09106969833374023
분노과 산책의 MAE : 0.09468939155340195
분노과 수면의 MAE : 0.0884054849545161
분노과 독서의 MAE : 0.14604143549998602
--------------------
분노에서 가장 유사한 item은 수면


우울과 명상의 MAE : 0.39506959915161133
우울과 운동의 MAE : 0.2877918630838394
우울과 산책의 MAE : 0.22182000925143563
우울과 수면의 MAE : 0.22521917025248211
우울과 독서의 MAE

In [27]:
# 감정과 가장 유사한 item을 추천 using 코사인 유사도
XYZ = emotion2item(action_est, emotion_est)
for i in range(0,5):
    XYZ.sim_item_Cosine(i)
    print('\n')


행복과 명상의 Cosine 유사도 : 0.955536527456638
행복과 운동의 Cosine 유사도 : 0.994872411534904
행복과 산책의 Cosine 유사도 : 0.9974026154686703
행복과 수면의 Cosine 유사도 : 0.9872293605064306
행복과 독서의 Cosine 유사도 : 0.9974811543503544
--------------------
행복에서 가장 유사한 item은 독서


열정과 명상의 Cosine 유사도 : 0.9614180257309098
열정과 운동의 Cosine 유사도 : 0.9970078018971505
열정과 산책의 Cosine 유사도 : 0.9976766463531613
열정과 수면의 Cosine 유사도 : 0.988836592355805
열정과 독서의 Cosine 유사도 : 0.9950727771467379
--------------------
열정에서 가장 유사한 item은 산책


두려움과 명상의 Cosine 유사도 : 0.7662237005871558
두려움과 운동의 Cosine 유사도 : 0.8736732143781738
두려움과 산책의 Cosine 유사도 : 0.9216419044232714
두려움과 수면의 Cosine 유사도 : 0.9274185392428561
두려움과 독서의 Cosine 유사도 : 0.9171433670042938
--------------------
두려움에서 가장 유사한 item은 수면


분노과 명상의 Cosine 유사도 : 0.9652905103518133
분노과 운동의 Cosine 유사도 : 0.9925318485281472
분노과 산책의 Cosine 유사도 : 0.991866593621276
분노과 수면의 Cosine 유사도 : 0.9913538207442711
분노과 독서의 Cosine 유사도 : 0.9778332795828822
--------------------
분노에서 가장 유사한 item은 운동


우울과 명상의 Cosine 유사도 : 0

In [28]:
# 감정과 가장 유사한 item을 추천 using 피어슨 유사도
XYZ = emotion2item(action_est, emotion_est)
for i in range(0,5):
    XYZ.sim_item_Pearson(i)
    print('\n')


행복과 명상의 피어슨 유사도 : 0.07925896969112922
행복과 운동의 피어슨 유사도 : 0.9403274446706629
행복과 산책의 피어슨 유사도 : 0.9510672482788439
행복과 수면의 피어슨 유사도 : 0.7262370008693378
행복과 독서의 피어슨 유사도 : 0.9749145074819885
--------------------
행복에서 가장 유사한 item은 독서


열정과 명상의 피어슨 유사도 : -0.02445318426215765
열정과 운동의 피어슨 유사도 : 0.8958800618176361
열정과 산책의 피어슨 유사도 : 0.9427398770879409
열정과 수면의 피어슨 유사도 : 0.6888263631099166
열정과 독서의 피어슨 유사도 : 0.9987445231159076
--------------------
열정에서 가장 유사한 item은 독서


두려움과 명상의 피어슨 유사도 : -0.4293823340731731
두려움과 운동의 피어슨 유사도 : 0.5797930543465886
두려움과 산책의 피어슨 유사도 : 0.9397302181375896
두려움과 수면의 피어슨 유사도 : 0.9893585454080616
두려움과 독서의 피어슨 유사도 : 0.7598333962450686
--------------------
두려움에서 가장 유사한 item은 수면


분노과 명상의 피어슨 유사도 : -0.5159936754590031
분노과 운동의 피어슨 유사도 : 0.5757108743509762
분노과 산책의 피어슨 유사도 : 0.9478392397287891
분노과 수면의 피어슨 유사도 : 0.9230324255172322
분노과 독서의 피어슨 유사도 : 0.8350508733010962
--------------------
분노에서 가장 유사한 item은 산책


우울과 명상의 피어슨 유사도 : 0.25264871959024665
우울과 운동의 피어슨 유사도 : 0.076576767705500

In [29]:
# 추청치가 가장 높은 차원 1개에 대해서 MAE를 이용해서 추천
XYZ = emotion2item(action_est, emotion_est)
for idx in range(0,5):
    XYZ.max_item_MAE(idx,1)
    print("\n\n")

행복의 MAX 차원 ['회복']
행복과 명상의 MAE : 0.027369817097981808
행복과 운동의 MAE : 0.027980923652648926
행복과 산책의 MAE : 0.01639656225840247
행복과 수면의 MAE : 0.12479899326960242
행복과 독서의 MAE : 0.03203725814819336
--------------------
행복에서 가장 유사한 item은 산책



열정의 MAX 차원 ['회복']
열정과 명상의 MAE : 0.2857420047124227
열정과 운동의 MAE : 0.230391263961792
열정과 산책의 MAE : 0.2747687498728434
열정과 수면의 MAE : 0.38317118088404334
열정과 독서의 MAE : 0.22633492946624756
--------------------
열정에서 가장 유사한 item은 독서



두려움의 MAX 차원 ['회복']
두려움과 명상의 MAE : 0.043725093205770005
두려움과 운동의 MAE : 0.011625647544860729
두려움과 산책의 MAE : 0.032751838366190666
두려움과 수면의 MAE : 0.14115426937739062
두려움과 독서의 MAE : 0.015681982040405162
--------------------
두려움에서 가장 유사한 item은 운동



분노의 MAX 차원 ['회복']
분노과 명상의 MAE : 0.10031394163767493
분노과 운동의 MAE : 0.15566468238830566
분노과 산책의 MAE : 0.11128719647725427
분노과 수면의 MAE : 0.0028847654660543176
분노과 독서의 MAE : 0.1597210168838501
--------------------
분노에서 가장 유사한 item은 수면



우울의 MAX 차원 ['에너지']
우울과 명상의 MAE : 0.11874788999557495
우울과 운

In [30]:
# 추청치가 가장 높은 차원 2개에 대해서 MAE를 이용해서 추천
XYZ = emotion2item(action_est, emotion_est)
for idx in range(0,5):
    XYZ.max_item_MAE(idx,2)
    print("\n\n")

행복의 MAX 차원 ['회복', '에너지']
행복과 명상의 MAE : 0.11405635873476666
행복과 운동의 MAE : 0.07848483324050903
행복과 산책의 MAE : 0.02816457549730933
행복과 수면의 MAE : 0.08103735248247779
행복과 독서의 MAE : 0.04198559125264484
--------------------
행복에서 가장 유사한 item은 산책



열정의 MAX 차원 ['회복', '에너지']
열정과 명상의 MAE : 0.20591090122858685
열정과 운동의 MAE : 0.21411260962486267
열정과 산책의 MAE : 0.28082942962646484
열정과 수면의 MAE : 0.3736347953478495
열정과 독서의 MAE : 0.25061185161272687
--------------------
열정에서 가장 유사한 item은 명상



두려움의 MAX 차원 ['회복', '정화']
두려움과 명상의 MAE : 0.04618081450462347
두려움과 운동의 MAE : 0.02905681729316706
두려움과 산책의 MAE : 0.04424315690994268
두려움과 수면의 MAE : 0.1290275057156881
두려움과 독서의 MAE : 0.053334663311640385
--------------------
두려움에서 가장 유사한 item은 운동



분노의 MAX 차원 ['회복', '정화']
분노과 명상의 MAE : 0.09219512343406677
분노과 운동의 MAE : 0.12094476819038391
분노과 산책의 MAE : 0.09413278102874756
분노과 수면의 MAE : 0.009348432223002134
분노과 독서의 MAE : 0.19171060125033063
--------------------
분노에서 가장 유사한 item은 수면



우울의 MAX 차원 ['에너지', '순환']
우울과 명상의 MA

In [31]:
# 추청치가 가장 높은 차원 3개에 대해서 MAE를 이용해서 추천
XYZ = emotion2item(action_est, emotion_est)
for idx in range(0,5):
    XYZ.max_item_MAE(idx,3)
    print("\n\n")

행복의 MAX 차원 ['회복', '에너지', '순환']
행복과 명상의 MAE : 0.27714234590530407
행복과 운동의 MAE : 0.11657500267028809
행복과 산책의 MAE : 0.0291329721609751
행복과 수면의 MAE : 0.08678974707921344
행복과 독서의 MAE : 0.045764939652548874
--------------------
행복에서 가장 유사한 item은 산책



열정의 MAX 차원 ['회복', '에너지', '정화']
열정과 명상의 MAE : 0.1876743170950148
열정과 운동의 MAE : 0.19385830561319986
열정과 산책의 MAE : 0.23525402281019422
열정과 수면의 MAE : 0.27673551109102035
열정과 독서의 MAE : 0.264016244146559
--------------------
열정에서 가장 유사한 item은 명상



두려움의 MAX 차원 ['회복', '정화', '에너지']
두려움과 명상의 MAE : 0.20403600732485458
두려움과 운동의 MAE : 0.16870195666948953
두려움과 산책의 MAE : 0.14914079838328895
두려움과 수면의 MAE : 0.1799275974432628
두려움과 독서의 MAE : 0.15920224785804746
--------------------
두려움에서 가장 유사한 item은 산책



분노의 MAX 차원 ['회복', '정화', '에너지']
분노과 명상의 MAE : 0.08589025338490804
분노과 운동의 MAE : 0.08113863070805867
분노과 산책의 MAE : 0.09193178680207996
분노과 수면의 MAE : 0.06114498774210612
분노과 독서의 MAE : 0.15298322174284196
--------------------
분노에서 가장 유사한 item은 수면



우울의 MAX 차원 ['

In [32]:
# 추청치가 가장 높은 차원 4개에 대해서 MAE를 이용해서 추천
XYZ = emotion2item(action_est, emotion_est)
for idx in range(0,5):
    XYZ.max_item_MAE(idx,1)
    print("\n\n")

행복의 MAX 차원 ['회복']
행복과 명상의 MAE : 0.027369817097981808
행복과 운동의 MAE : 0.027980923652648926
행복과 산책의 MAE : 0.01639656225840247
행복과 수면의 MAE : 0.12479899326960242
행복과 독서의 MAE : 0.03203725814819336
--------------------
행복에서 가장 유사한 item은 산책



열정의 MAX 차원 ['회복']
열정과 명상의 MAE : 0.2857420047124227
열정과 운동의 MAE : 0.230391263961792
열정과 산책의 MAE : 0.2747687498728434
열정과 수면의 MAE : 0.38317118088404334
열정과 독서의 MAE : 0.22633492946624756
--------------------
열정에서 가장 유사한 item은 독서



두려움의 MAX 차원 ['회복']
두려움과 명상의 MAE : 0.043725093205770005
두려움과 운동의 MAE : 0.011625647544860729
두려움과 산책의 MAE : 0.032751838366190666
두려움과 수면의 MAE : 0.14115426937739062
두려움과 독서의 MAE : 0.015681982040405162
--------------------
두려움에서 가장 유사한 item은 운동



분노의 MAX 차원 ['회복']
분노과 명상의 MAE : 0.10031394163767493
분노과 운동의 MAE : 0.15566468238830566
분노과 산책의 MAE : 0.11128719647725427
분노과 수면의 MAE : 0.0028847654660543176
분노과 독서의 MAE : 0.1597210168838501
--------------------
분노에서 가장 유사한 item은 수면



우울의 MAX 차원 ['에너지']
우울과 명상의 MAE : 0.11874788999557495
우울과 운